In [190]:
import re
import emoji
import random
import pandas as pd
from pandas import read_csv
import os, itertools
from itertools import groupby

In [191]:
# load data
map_ = read_csv('Map_Ramen_data.csv', encoding='utf8',index_col=0)
fb_ = read_csv('store_id_done.csv', encoding='utf8',index_col=0)
fb_ = fb_.drop(columns=['store_id','post_id'])
map_ = map_.rename(columns = {'discription': 'description', 'mapReview': 'map_review', 'openTime': 'open_time'})
map_

,store,address,description,open_time,latitude,longtitute,map_review,region,soup,transport
store_id,,,,,,,,,,
1,雞吉君拉麵とりよしくんラーメン雞白湯專門店,114台北市內湖區環山路一段33號,雞吉君是雞湯愛好者必朝聖的一間店，口味上從清淡到濃厚分別是淡麗->濃厚->特濃，三種口味各有...,週一 店休 PS：⚠️適逢每月雙數週休一、二週二~四 17:30~21:00(或售完)週五~...,25.086283,121.566969,Zhanyao Zhang-濃厚煮干雞拉麵 https://bit.ly/2QmFNoJ...,北,#ramentw #台灣拉麵愛好會 #拉麵 #油拌麵 #淡麗 #雞白 #雞濃 #雞清 #煮干...,西湖國小／捷運西湖站
2,勝王,104台北市中山區林森北路306號,2020年8月22日重新開幕，本店為台灣第一個二毛作(原意為同一塊耕地一年內種植不同作物，在...,11:30~14:00、17:00~20:30,25.056086,121.525334,鄭傑瑞 - 橘醋清雞湯拉麵 https://bit.ly/38lDEnC 侯懿純 -...,北,#ramentw #台灣拉麵愛好會 #拉麵 #雞白湯 #薩索雞 #變化 #2019金賞,金光鐘錶
3,五之神製作所 台灣,110台北市信義區忠孝東路四段553巷6弄6號,為拉麵界較為少見的，以蝦作為基底的拉麵店，一共分為豚骨蝦湯麵、蝦沾麵與番茄蝦沾麵，其叉燒飯則...,週一~五 11:30~15:00、17:00~21:00週二 公休週六~日 11:30~21:00,25.042253,121.563041,Michael Lin - 雞白湯拉麵 https://bit.ly/360ZBqo ...,北,#ramentw #台灣拉麵愛好會 #拉麵 #沾麵 #豚骨 #豚骨魚介 #蝦豚骨 #限定 #...,松菸／捷運市政府站 - 1號出口
4,Okaeriお帰り吃碗拉麵吧,106台北市大安區延吉街60號,年休中，10月26日(一)11:30再開。開拓台灣純正日本口味拉麵疆土的老店之一、與雞吉君為...,週二~週日: 11:30～15:00或（售完為止）17:30～21:00或（售完為止）每週一...,25.043711,121.553755,Queena Peng - 濃厚豚骨拉麵／辛口豚骨拉麵 https://bit.ly/3...,北,#ramentw #台灣拉麵愛好會 #拉麵 #豚骨 #豚骨魚介 #辣味 #煎餃 #2019金賞,華視／捷運國父紀念館站
5,隱家拉麵-士林店,111台北市士林區中山北路五段505巷1號,開業於2019年10月，本店的風格為多樣化的湯頭，目前提供雞湯與豚骨湯二種湯頭，以及濃厚蝦沾...,11:30~14:30、16:30~22:00,25.093249,121.527470,風月 - 辛豚骨拉麵 https://bit.ly/37Ljcfw 高晉 - 黃金雞...,北,#ramentw #台灣拉麵愛好會 #拉麵 #沾麵 #豚骨 #雞清湯 #豚骨蝦 #限定 #加...,捷運士林站
6,隱家拉麵-赤峰店,104台灣台北市大同區,開幕於2020年5月，本店為隱家拉麵二店，主打黃金雞湯、辛豚骨，並有每日限定擔擔麵、鯛白湯沾...,11:30~14:00、16:30~22:00,23.975744,120.979772,風月 - 辛豚骨拉麵 https://bit.ly/37Ljcfw 高晉 - 黃金雞...,北,#ramentw #台灣拉麵愛好會 #拉麵 #雞清湯 #豚骨 #魚白湯 #擔擔麵 #加麵一球免費,捷運雙連站
7,柑橘 Shinn,106台北市大安區仁愛路四段228-6號,開業於2017年3月，融合美日台三種文化風格融合的油拌麵，面對的族群以女性客人及上班族居多，...,週一~五 11:30 - 14:00、17:00 - 20:30週六~日 11:30 - 2...,25.037301,121.552133,Jimmy Lai - 柑橘濃厚雞白湯 https://bit.ly/3it4hss ...,北,#ramentw #台灣拉麵愛好會 #拉麵 #油拌麵 #柑橘 #雞白 #雞濃 #雞清 #淡麗...,仁愛路國中步行2分／捷運信義安和站
8,山嵐拉麵-台灣總店,106台北市大安區杭州南路二段75號,於2019年6月遷移至現址，店內所售品項分為白湯(豚骨)、赤湯(加辣)、海湯(加魚粉)，三種...,11:30~14:30、17:15~20:45,25.029168,121.522079,姿婷王 - 白湯豚骨／赤湯豚骨 https://bit.ly/3689RvY Is ...,北,#ramentw #台灣拉麵愛好會 #拉麵 #沾麵 #豚骨 #豚骨魚介 #辣味 #2019金賞,南門市場 / 捷運古亭站步行4分
9,山嵐拉麵-公館店,100台北市中正區羅斯福路四段136巷1弄13號,在東區的分店開設後，山嵐決定於公館開設第二間店，本店除了擁有與忠孝店相同的定番外，更常常推出...,週日~周四: 11:30~21:00周五~周六: 11:30~21:30,25.013039,121.535375,姿婷王 - 白湯豚骨／赤湯豚骨 https://bit.ly/3689RvY Is ...,北,#ramentw #台灣拉麵愛好會 #拉麵 #沾麵 #豚骨 #豚骨魚介 #辣味 #2019金賞,台大／捷運公館站


In [192]:
map_ = map_.sort_values(by=['store'])
map_ = map_.reset_index(drop=True)
map_ = map_.sort_index(axis=0 ,ascending=True)
map_ = map_.iloc[::-1]
map_ = map_.reset_index(drop=True)
#map_

In [193]:
### Classify MAIN or DETAILED store
map_store = list(map_['store'])
detail_store = []
stem_detail_store = []

for s in map_store:
    store = s.lower()
    store = store.replace('（','(').replace('）',')').replace(u'\u3000',u'').replace('台','臺').replace('-','#')
    if ('高雄夢時代店' in store) or ('林口三井店' in store) or ('樂利店' in store) or ('臺中一中店' in store) or  \
            ('土城日月光店' in store) or ('中山店' in store) or ('高雄五福店' in store) or ('臺中逢甲店' in store) or  \
            ('師大店' in store) or ('竹北店' in store) or ('延吉街店' in store) or ('成功店' in store) or  \
            ('臺北凱撒店' in store) or ('高鐵店' in store) or ('臺中清水店' in store) or ('家樂福店' in store) or  \
            ('新光中港店' in store) or ('臺北信義店' in store) or ('臺中中友店' in store) or ('新竹勝利店' in store) or  \
            ('公館店' in store) or ('一中分店' in store) or ('大潤發店' in store) or ('小北店' in store) or  \
            ('新光三越' in store) or ('微風南山' in store):
        detail_store.append(s)
        stem_detail_store.append(s[:-3])
    elif ('express' in store):
        detail_store.append(s)
        stem_detail_store.append(s)
    else:
        if ('(' in store) and (')' in store) and (store[store.index(')')-1] == '店') \
            and ('專賣' not in store) and ('專門' not in store) and ('本店' not in store) and ('總店' not in store):
            #print(store)
            detail_store.append(s)
            stem_detail_store.append(s[:-4])
        elif ('#' in store) and ('店' in store[store.index('#'):]) and ('本店' not in store) \
            and ('專門' not in store) and ('本店' not in store) and ('總店' not in store):
            detail_store.append(s)
            stem_detail_store.append(s[:-4])
            #print(store)
        elif ('#' in store) and ('街' in store[store.index('#'):]) and '專賣' not in store \
            and ('專門' not in store) and ('本店' not in store) and ('總店' not in store):
            detail_store.append(s)
            stem_detail_store.append(s[:-4])
            #print(store)  
        elif len(store) >= 5 and (' ' == store[-4]) and ('店'== store[-1]):
            detail_store.append(s) 
            stem_detail_store.append(s[:-3])
        elif ('組' in store[-1]):
            detail_store.append(s)
            stem_detail_store.append(s[:-3])
        elif ('分店' in store) and ('臺灣分店' not in store):
            detail_store.append(s)      
            stem_detail_store.append(s[:-3])

#detail_store

main_store = list(set(map_store) - set(detail_store))
print(len(main_store)) #285
stem_detail_store_ = stem_detail_store
print(len(stem_detail_store_)) #146
stem_detail_store = list(set(stem_detail_store))
all_main_store = list(set(main_store + stem_detail_store))
print(len(set(map_store)))
main_store

285
146
431


['三元堂拉麵專門店',
 '瀧澤軒食堂［拉麵屋］',
 'Kido拉麵',
 '北海道札幌拉麵羽畠食堂',
 '勝拉麵*',
 '麵屋北辰 めん処北辰',
 '墨洋拉麵',
 '豚骨醬油拉麵 初',
 '麵屋十八甲',
 '蝦蝦拉麵 元氣屋',
 '屋台拉麵',
 '二月ラーメン',
 '無常拉麵',
 '樂山窄巷拉麵',
 '宮本九州天然豚骨拉麵専門店',
 '均晣溫度',
 '大角拉麵 ダージャオラーメン',
 '晴拉麵',
 'OSHO拉麵',
 '一前一町拉麵',
 '渡山樓',
 '田中そば',
 '宮內屋豚骨らーめん',
 '麵屋一燈 Menya Itto Taiwan',
 '玄一拉麵 GENICHI RAMEN',
 '餃 麵',
 '道樂商店',
 '織田信長',
 '小卷拉麵屋',
 '双豚ラーメン',
 '麵屋祥 東京醬油拉麵',
 '創作麵坊 鮭の大助',
 '柑橘 Shinn',
 '京都柚子豚骨拉麵研究中心',
 '百八龍拉麵',
 '濟善老麵',
 '麵屋柴 雞白湯拉麵專賣店',
 '辰拉麵',
 '裕手串·拉麵',
 '麵屋壹慶',
 '麵屋豚彩',
 '許諺屋',
 '鳳華拉麵 （らぁ麺 鳳華）',
 '哈拉麵',
 '一蘭拉麵-台灣台北本店別館',
 '川又屋日式拉麵',
 '勝王',
 '麵屋高',
 '半熟堂 Hanjukudo Taiwan',
 '松食堂',
 '金沢拉麵',
 '李好拉麵-大吉大利-今日吃拉麵',
 '初原麵場-三峽本店',
 '麵屋輝',
 '蒔桐日式拉麵',
 '一心拉麵',
 '太子豚骨ラーメン',
 '初 拉麵&煎餃',
 '幸花拉麵雞白湯專門店',
 '荷麵亭 Hasumentei',
 '麗人博多拉麵-板橋本店',
 '惠比須麵屋',
 '麵屋秋鳴',
 '豚人ラ－メン-敦化本店',
 '小川拉麺専門店 台灣分店',
 '宜蘭鬼匠拉麵-總店',
 '狸匠拉麵-深夜拉麵',
 '激勵軒 Chicken Power Ramen',
 '京之林拉麵',
 '東京醬油拉麵超極丸',
 '次元食堂',
 '東京豚骨 釜本舖',
 '熊越岳',
 '百壽爺日式拉麵專門店',
 'Mr.RAMEN 豚骨拉麵店',
 '北一家 食事処',
 '中村製麵',
 '佐原拉麵',
 '希ラーメン拉麵',
 '火曜

In [194]:
#len(detail_store) #  145
#len(stem_detail_store) # 145 -> 125
#len(main_store) # 286
#len(all_main_store) # 408
#detail_store

In [195]:
map_['tag'] = map_.store.isin(detail_store)
#df[df.E.isin(['a','c'])]
#map_.store.isin()
map_detail = map_.loc[map_['tag'] == True]
#df.loc[df['B'].isin(['one','three'])

map_main = map_.loc[map_['tag'] == False]
second_main_store = list(map_main['store'])

In [196]:
map_store_lst = []

for s in all_main_store:
    store = re.sub(r'[^\w\s]','',s)
    store = store.replace('総','總').replace('麺','麵').replace('醤','醬').replace('ichi','ICHI').replace('番茄','蕃茄')  \
                    .replace('#','').replace(' ','').replace(u'\u3000',u'').replace('太陽蕃茄EXPRESS','太陽蕃茄拉麵EXPRESS') \
                    .replace('《','').replace('》','').replace('台','臺').replace('らーめん','拉麵').replace('の','之') \
                    .replace('。','').replace('、',' ').replace('｜','').replace('？','').replace('ラーメン','拉麵') \
                    .replace('(','').replace(')','').replace('·','').replace('/','').replace('aqua2','').replace('麵鋪','麵舖') \
                    .replace('※','').replace('ぁ','あ').replace('²','2').replace('台湾','臺灣')                
    map_store_lst.append(store) 
shop_sorted = sorted(map_store_lst)

detail_map_store_lst = []
for s in stem_detail_store_:
    store = re.sub(r'[^\w\s]','',s)
    store = store.replace('総','總').replace('麺','麵').replace('醤','醬').replace('ichi','ICHI').replace('番茄','蕃茄')  \
                    .replace('#','').replace(' ','').replace(u'\u3000',u'').replace('太陽蕃茄EXPRESS','太陽蕃茄拉麵EXPRESS') \
                    .replace('《','').replace('》','').replace('台','臺').replace('らーめん','拉麵').replace('の','之') \
                    .replace('。','').replace('、',' ').replace('｜','').replace('？','').replace('ラーメン','拉麵') \
                    .replace('(','').replace(')','').replace('·','').replace('/','').replace('aqua2','').replace('麵鋪','麵舖') \
                    .replace('※','').replace('ぁ','あ').replace('²','2').replace('台湾','臺灣')                
    detail_map_store_lst.append(store) 

second_main_store_lst = []    
for s in list(second_main_store):
    store = re.sub(r'[^\w\s]','',s)
    store = store.replace('総','總').replace('麺','麵').replace('醤','醬').replace('ichi','ICHI').replace('番茄','蕃茄')  \
                    .replace('#','').replace(' ','').replace(u'\u3000',u'').replace('太陽蕃茄EXPRESS','太陽蕃茄拉麵EXPRESS') \
                    .replace('《','').replace('》','').replace('台','臺').replace('らーめん','拉麵').replace('の','之') \
                    .replace('。','').replace('、',' ').replace('｜','').replace('？','').replace('ラーメン','拉麵') \
                    .replace('(','').replace(')','').replace('·','').replace('/','').replace('aqua2','').replace('麵鋪','麵舖') \
                    .replace('※','').replace('ぁ','あ').replace('²','2').replace('台湾','臺灣')                
    second_main_store_lst.append(store)  
    

In [197]:
second_main_store_lst
map_detail['stem_store'] = detail_map_store_lst
map_main['stem_store'] = second_main_store_lst
map_t2 = pd.concat([map_main, map_detail], sort=True)
map_t2

/Users/yuchiacheng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/yuchiacheng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,address,description,latitude,longtitute,map_review,open_time,region,soup,stem_store,store,tag,transport
0,無提供地址資訊,雖是實體店面，仍以屋台風格設計店內有許多日系元素裝潢，猶如置身世外,NaN,NaN,吳俊青 - 塩味豚骨ラーメン http://bit.ly/2wbxeTk Lily ...,18:00~02:00,中,#ramentw #台灣拉麵愛好會 #拉麵 #豚骨,龍魂屋臺拉麵,龍魂屋台ラーメン,False,僑泰高中
1,970花蓮縣花蓮市復興街71號,開幕於2017年11月，2020年5月1日遷移至現址，是花蓮在地首屈一指的拉麵專門店，深受在...,23.981691,121.606582,Youmu K YK - 黑蒜拉麵 https://bit.ly/39o2aVm 林...,11:30~13:30、17:30~20:10,東,#ramentw #台灣拉麵愛好會 #拉麵 #豚骨 #炸串,黑潮拉麵炸串專賣店,黑潮拉麵·炸串專賣店,False,花蓮市戶政事務所
2,112台北市北投區尊賢街206巷2號,開業於2019年11月，以豚骨湯為主體的拉麵店，味道屬於台式風格，叉燒肉為此店的特色，詳情可...,25.115297,121.508182,由你來寫第一篇,11:00~21:00,北,#ramentw #台灣拉麵愛好會 #拉麵 #豚骨,黑曜麵屋,黑曜麵屋,False,曹家刈包
3,265宜蘭縣羅東鎮南門路158號,開幕於2015年6月，深耕宜蘭，屬於全台知名的在地拉麵店家，部份麵友們暱稱本店為「宜蘭王」「...,24.673032,121.764848,蔡承澔 - 鹽味魚介薩索雞白湯 https://bit.ly/3fOPwyB Sta...,週二~五：12:00–14:30、17:00–20:20週六~日：12:00–20:20,東,#ramentw #台灣拉麵愛好會 #拉麵 #豚骨 #2019名店,麵魂家,麵魂家,False,羅東文化廣場對面
4,105台北市松山區健康路68號之3號,開幕於2020年1月16日，主軸為雞湯味，詳情可察看下方網友食記。,25.053285,121.553543,徐煜傑 - 鹽味黃金雞湯麵 https://bit.ly/2NwzPA7 Jimmy...,11:30~14:30 、17:30~20:30週六、日 不定休,北,#ramentw #台灣拉麵愛好會 #拉麵 #雞湯,麵試十一次,麵試十一次,False,胖老爹健康路店
7,406台中市北屯區熱河路二段210號,開業於2019年11月，是狸匠系列的第四家店，主打魚介系湯頭風味，亦會不時推出新限定，詳情可...,24.174131,120.688134,吳俊青 - 清酒鮭魚 https://bit.ly/39aJloN Yo Lin -...,11:30-14:00、17:30-21:00,中,#ramentw #台灣拉麵愛好會 #拉麵 #沾麵 #魚介 #泡系 #限定 #變化,麵本初純水魚貝拉麵,麵本初 純水魚貝拉麵,False,民俗公園
8,320桃園市中壢區力行北街50號,於2020年1月11日開業，原有豚骨與雞白湯系，目前以雞白湯為主軸，詳情可察看下方網友食記。,24.954455,121.239747,李奇來 - 雞白湯拉麵、泡系雞湯 https://bit.ly/2x9tdTH 陳柏...,17:30~21:00,北,#ramentw #台灣拉麵愛好會 #拉麵 #雞白,麵心無界,麵心無界,False,力行北街
9,800高雄市新興區六合一路137號,開業於2019年5月，以豚骨湯為主軸，亦有販售泡系拉麵，詳情可察看下方網友食記。,22.632690,120.303868,寒沐蝶 - 地獄叉燒拉麵 https://bit.ly/2EvAyQU Syu Ze...,週一-五 17:00–21:00週六-日 11:00–14:00、17:00–21:00週四 公休,南,#ramentw #台灣拉麵愛好會 #拉麵 #豚骨,麵屋高,麵屋高,False,華大商旅／六合一路
10,71081台南市永康區新行街33巷1號,開業於2019年11月，以豚湯系為主體，近來不定時會開發新品項，以饗麵友們的味蕾，詳情可察看...,23.039053,120.241287,Huang Wei - 限定。濃厚虱目魚白湯 https://bit.ly/35Apzk...,11:30~13:30、17:00~20:00週四 公休,南,#ramentw #台灣拉麵愛好會 #拉麵 #豚骨 #擔擔麵,麵屋青鈴麵や青鈴,麵屋青鈴（麺や青鈴）,False,台南應用科大
11,24152新北市三重區仁愛街688號,開業於2019年10月，是道樂屋台所開設的第三家店舖，詳情可察看下方網友食記。,25.080212,121.481548,Stars Yang - 味噌拉麵 https://bit.ly/2MrE3uc F...,12:00–14:00、17:00–21:00(售完為止)週一 公休,北,#ramentw #台灣拉麵愛好會 #拉麵 #豚湯 #2019名店,麵屋道樂,麵屋道樂,False,仁愛街


In [198]:
### FIRST GROUPING
util_func = lambda x: x[0]  
first_sort = [list(ele) for i, ele in groupby(shop_sorted, util_func)] 
lst = list(itertools.chain(*first_sort))
#first_sort

In [199]:
### SECOND GROUPING
unique = []
second_sort = []

util_func = lambda x: x[1]
for item in first_sort:
    if len(item) > 1:
        temp = sorted(item, key = util_func)
        second_sort.append([list(ele) for i, ele in groupby(temp, util_func)])
        #second_sort.append([list(g) for _, g in itertools.groupby(item, lambda x: x[1])])
    else:
        unique.append(item[0])

second_sort_len = []

for item_1 in second_sort:
    for item_2 in item_1: 
        if len(item_2) > 1:
            item_2 = sorted(item_2, key=len)
            second_sort_len.append(item_2)
        else:
            unique.append(item_2[0])
unique

['11番町豚骨拉麵',
 'Dueitalian',
 'Nani麵',
 'あお実家和食処另一個家',
 'いけっ麵拉麵専門店ikemanramen',
 'つけ麵心心',
 'ひろ麵屋',
 '不二家拉麵',
 '二月拉麵',
 '今鶏制湯所',
 '伍拉麵',
 '佐原拉麵',
 '信川屋博多豚骨拉麵店',
 '俺之豚',
 '倉麵屋拉麵',
 '全粒粉濃厚豚骨拉麵拉麵壱RamenIchi',
 '六丁目拉麵',
 '匠麵殿拉麵專門店',
 '十劍拉麵',
 '千勝軒拉麵',
 '半熟堂HanjukudoTaiwan',
 '双豚拉麵',
 '同拉麵',
 '味之時計臺',
 '品麵屋拉麵',
 '哈拉麵',
 '嘿傑克',
 '圓拉麵',
 '坂道拉麵',
 '均晣溫度',
 '埔里橫濱拉麵',
 '墨洋拉麵',
 '嬡京都拉麵',
 '定置漁場三代目TheFishery',
 '宜蘭鬼匠拉麵總店',
 '屋臺拉麵',
 '希拉麵拉麵',
 '幸花拉麵雞白湯專門店',
 '彩日式拉麵專賣店',
 '御嶽海拉麵ラㄧメン',
 '惠比須麵屋',
 '招福製麵所拉麵',
 '拾貳麵舖',
 '晴拉麵',
 '暴走中村',
 '有囍yoshiramenよし拉麵',
 '木村Kimura',
 '李好拉麵大吉大利今日吃拉麵',
 '松食堂',
 '柑橘Shinn',
 '梅之香拉麵',
 '森本家拉麵',
 '極清拉麵KiyomeRamen',
 '横浜家系拉麵拉麵家',
 '次元食堂',
 '沐瀧家拉麵十二月預定',
 '泰拉風泰式拉麵',
 '涼太拉麵拉麵涼太',
 '淳鳩一夫拉麵',
 '淺草咖哩蛋包',
 '渡山樓',
 '激勵軒ChickenPowerRamen',
 '濃氣屋拉麵',
 '濟善老麵',
 '瀧澤軒食堂拉麵屋',
 '熱烈一番拉麵',
 '燈來拉麵',
 '燒燒一碗來',
 '狸匠拉麵深夜拉麵',
 '獅神拉麵',
 '琉球山日式拉麵店',
 '田中そば',
 '男子漢拉麵食堂內湖',
 '盡心亭Heartsoul',
 '目黑拉麵',
 '真劍拉麵',
 '研田拉麵',
 '竹拉麵関東清爽系豚骨',
 '籽田野菜屋',
 '織田信長',
 '美濃屋拉麵',
 '翔大拉麵店',
 '臥龍拉麵',
 '荷麵亭Hasumentei',
 '菜良

In [200]:
### THIRD GROUPING
third_sort = []

util_func = lambda x: x[2]
for item in second_sort_len:
    if item[0][:2] == '山嵐' or item[0][:3] == '赤麵廠' or item[0][:3] == '花月嵐' or item[0][:4] == '麵屋一燈'or item[0][:3] == '麵屋濃' \
        or item[0][:4] == '拉麵二郎' or item[0][:3] == '一風堂' or item[0][:4] == '鳥人拉麵' or item[0][:5] == '滝禾製麵所'\
        or item[0][:4] == '神山拉麵'or item[0][:4] == '赤坂拉麵' or item[0][:11] == '鷹流東京醬油拉麵蘭丸':
        print(item)
        third_sort.append([item])
    else:
        if len(item) > 1 and len(item[0]) >= 3:
            temp = sorted(item, key = util_func)
            third_sort.append([list(ele) for i, ele in groupby(temp, util_func)])
            #third_sort.append([list(g) for _, g in itertools.groupby(item, lambda x: x[2])])
        elif len(item) > 1 and len(item[0]) <= 2:
            third_sort.append([item])
        else:
            unique.append(item[0])
        
third_sort_len = []

for item_1 in third_sort:
    for item_2 in item_1: 
        if len(item_2) > 1:
            item_2 = sorted(item_2, key=len)
            third_sort_len.append(item_2)
        else:
            unique.append(item_2[0])

['一風堂微風', '一風堂桃園', '一風堂微風臺', '一風堂中山本店', '一風堂新光三越店', '一風堂EXPRESS微', '一風堂EXPRESS新竹巨城店']
['山嵐拉麵', '山嵐拉麵', '山嵐拉麵一', '山嵐拉麵臺灣總店']
['滝禾製麵所內', '滝禾製麵所太', '滝禾製麵所苗']
['神山拉麵', '神山拉麵彰', '神山拉麵高']
['花月嵐信', '花月嵐松', '花月嵐西', '花月嵐臺北', '花月嵐內湖大', '花月嵐新店家', '花月嵐重新家', '花月嵐拉麵新光', '花月嵐拉麵臺中']
['赤麵廠', '赤麵廠新莊總店']


In [201]:
### FOURTH GROUPING
fourth_sort = []

util_func = lambda x: x[3]
for item in third_sort_len:
    if item[0][:2] == '山嵐' or item[0][:3] == '赤麵廠' or item[0][:4] == '麵家三士' or item[0][:3] == '花月嵐' or item[0][:4] == '麵屋一燈'\
        or item[0][:4] == '拉麵二郎' or item[0][:3] == '一風堂' or item[0][:4] == '鳥人拉麵' or item[0][:11] == '鷹流東京醬油拉麵蘭丸' or item[0][:3] == '麵屋濃' \
        or item[0][:4] == '神山拉麵'or item[0][:4] == '赤坂拉麵' or item[0][:4] == '力量拉麵' or item[0][:4]== '京正拉麵'or item[0][:5] == '滝禾製麵所':
        fourth_sort.append([item])    
        print(item)
    else:
        if len(item) > 1 and len(item[0]) >= 4:
            temp = sorted(item, key = util_func)
            fourth_sort.append([list(ele) for i, ele in groupby(temp, util_func)])        
        elif len(item) > 1 and len(item[0]) <= 3:
            fourth_sort.append([item])
        else:
            unique.append(item[0])
        
fourth_sort_len = []

for item_1 in fourth_sort:
    for item_2 in item_1: 
        if len(item_2) > 1:
            item_2 = sorted(item_2, key=len)
            fourth_sort_len.append(item_2)
        else:
            unique.append(item_2[0])

['一風堂微風', '一風堂桃園', '一風堂微風臺', '一風堂中山本店', '一風堂新光三越店', '一風堂EXPRESS微', '一風堂EXPRESS新竹巨城店']
['力量拉麵', '力量拉麵仁愛']
['山嵐拉麵', '山嵐拉麵', '山嵐拉麵一', '山嵐拉麵臺灣總店']
['滝禾製麵所內', '滝禾製麵所太', '滝禾製麵所苗']
['神山拉麵', '神山拉麵彰', '神山拉麵高']
['花月嵐信', '花月嵐松', '花月嵐西', '花月嵐臺北', '花月嵐內湖大', '花月嵐新店家', '花月嵐重新家', '花月嵐拉麵新光', '花月嵐拉麵臺中']
['赤麵廠', '赤麵廠新莊總店']
['麵家三士', '麵家三士桃', '麵家三士土城日']
['麵屋濃三', '麵屋濃林口']


In [202]:
### FIFTH GROUPING
fifth_sort = []

util_func = lambda x: x[4]
for item in fourth_sort_len:
    if item[0][:2] == '山嵐' or item[0][:3] == '赤麵廠' or item[0][:4] == '麵家三士' or item[0][:3] == '花月嵐' or item[0][:4] == '麵屋一燈' \
        or item[0][:4] == '拉麵二郎' or item[0][:3] == '一風堂' or item[0][:4] == '鳥人拉麵' or item[0][:11] == '鷹流東京醬油拉麵蘭丸' or item[0][:3] == '麵屋濃'\
        or item[0][:4] == '神山拉麵'or item[0][:4] == '赤坂拉麵' or item[0][:4] == '力量拉麵' or item[0][:4]== '京正拉麵' or item[0][:5] == '滝禾製麵所':
        fifth_sort.append([item])    
    else:
        if len(item) > 1 and len(item[0]) > 7:
            temp = sorted(item, key = util_func)
            fifth_sort.append([list(ele) for i, ele in groupby(temp, util_func)])        
        elif len(item) > 1 and len(item[0]) <= 4:
            fifth_sort.append([item])            
        elif len(item) > 1 and len(item[0]) >= 5 and len(item[0]) <= 7:
            fifth_sort.append([item])
        else:
            unique.append(item[0])
            

fifth_sort_len = []

for item_1 in fifth_sort:
    for item_2 in item_1: 
        if len(item_2) > 1:
            item_2 = sorted(item_2, key=len)
            fifth_sort_len.append(item_2)
        else:
            unique.append(item_2[0])
fifth_sort_len

[['Hirosらあ麵Kitchen', 'Hirosらあ麵Kitchen', 'Hirosらあ麵Kitchen新'],
 ['LeMing麵食精釀餐酒館', 'LeMing麵食精釀餐酒館'],
 ['Mr拉麵雲', 'Mr拉麵絆臺南', 'Mr拉麵之野崎家'],
 ['Ramen初', 'RamenIzakaya空海拉麵'],
 ['TottoRamen鳥人拉麵', 'TottoRamen鳥人拉麵'],
 ['一幻拉麵臺北', '一幻拉麵x胡同居酒屋'],
 ['一本拉麵梟', '一本拉麵本店'],
 ['一蘭拉麵臺灣臺北本店', '一蘭拉麵臺灣臺北本店別館'],
 ['一風堂微風',
  '一風堂桃園',
  '一風堂微風臺',
  '一風堂中山本店',
  '一風堂新光三越店',
  '一風堂EXPRESS微',
  '一風堂EXPRESS新竹巨城店'],
 ['中華そば寶來軒', '中華そば太郎拉麵'],
 ['元町家橫濱家系拉麵', '元町家橫濱家系拉麵'],
 ['力量拉麵', '力量拉麵仁愛'],
 ['叢林拉麵', '叢林拉麵'],
 ['太陽蕃茄拉麵',
  '太陽蕃茄拉麵',
  '太陽蕃茄拉麵左',
  '太陽蕃茄拉麵漢',
  '太陽蕃茄拉麵誠',
  '太陽蕃茄拉麵大葉',
  '太陽蕃茄拉麵站前本店',
  '太陽蕃茄拉麵EXPRESS'],
 ['奧特拉麵RamenUltra南', '奧特拉麵RamenUltra板'],
 ['富士達人日本拉麵', '富士達人日本拉麵東寧總店'],
 ['屯京拉麵', '屯京拉麵', '屯京拉麵忠', '屯京拉麵臺', '屯京拉麵林口', '屯京拉麵高雄夢', '屯京拉麵新光三越'],
 ['山嵐拉麵', '山嵐拉麵', '山嵐拉麵一', '山嵐拉麵臺灣總店'],
 ['山形心心', '山形心心拉麵', '山形心心拉麵嘉', '山形心心拉麵高雄', '山形心心總店臺中', '山形心心拉麵臺北站'],
 ['山頭火拉麵', '山頭火拉麵sogo復'],
 ['東京油組總本店臺北', '東京油組總本店臺北'],
 ['東京豚骨釜本舖', '東京豚骨拉麵之野崎家'],
 ['樂麵屋', '樂麵屋南', '樂麵屋永'],
 ['橫濱家系拉麵大和家', '橫濱家系拉麵特濃屋'],
 ['滝禾製麵所內', '滝禾製

In [203]:
done = []
unclassified_group = []

for item in fifth_sort_len:
    grouped = []
    unclassified = []
    grouped.append(item[0])
    if item[0][:3] == '一風堂' or item[0][:4] == '麵屋一燈':
        done.append(item)
    elif len(item) <= 3 and len(item[0]) <= 6 and len(item[1]) <= 6:
        done.append(item)
    else:
        for i in range(len(item)-1):
            if (item[0] in item[i+1]) or (item[0][:6] in item[i+1]):
                grouped.append(item[i+1])
            else:
                unclassified.append(item[i+1])
        done.append(grouped)
        if unclassified != []:
            unclassified_group.append(unclassified)
unclassified_group 

[['Mr拉麵絆臺南', 'Mr拉麵之野崎家'],
 ['RamenIzakaya空海拉麵'],
 ['一幻拉麵x胡同居酒屋'],
 ['中華そば太郎拉麵'],
 ['東京豚骨拉麵之野崎家'],
 ['花月嵐松', '花月嵐西', '花月嵐臺北', '花月嵐內湖大', '花月嵐新店家', '花月嵐重新家', '花月嵐拉麵新光', '花月嵐拉麵臺中']]

In [204]:
for item in unclassified_group:
    if len(item) != 1 and item[0] != 'Mr拉麵絆臺南':
        temp_grouped = []
        temp_unclassified = []
        temp_grouped.append(item[0])
        for i in range(len(item)-1):
            if (item[0] in item[i+1]) or (item[0][:len(item[0])//2] in item[i+1]):
                temp_grouped.append(item[i+1])
            else:
                temp_unclassified.append(item[i+1])
        done.append(temp_grouped)
        #print(temp_unclassified)
        if temp_unclassified != [] and len(temp_unclassified) == 1:
            unique.append(temp_unclassified[0])
        elif temp_unclassified != [] and len(temp_unclassified) > 1:
            unique.append(temp_unclassified[:len(temp_unclassified)])
    elif len(item) != 1 and item[0] == 'Mr拉麵絆臺南':
        unique.append(item[0])
        unique.append(item[1])
    else:
        unique.append(item[0])
unique

['11番町豚骨拉麵',
 'Dueitalian',
 'Nani麵',
 'あお実家和食処另一個家',
 'いけっ麵拉麵専門店ikemanramen',
 'つけ麵心心',
 'ひろ麵屋',
 '不二家拉麵',
 '二月拉麵',
 '今鶏制湯所',
 '伍拉麵',
 '佐原拉麵',
 '信川屋博多豚骨拉麵店',
 '俺之豚',
 '倉麵屋拉麵',
 '全粒粉濃厚豚骨拉麵拉麵壱RamenIchi',
 '六丁目拉麵',
 '匠麵殿拉麵專門店',
 '十劍拉麵',
 '千勝軒拉麵',
 '半熟堂HanjukudoTaiwan',
 '双豚拉麵',
 '同拉麵',
 '味之時計臺',
 '品麵屋拉麵',
 '哈拉麵',
 '嘿傑克',
 '圓拉麵',
 '坂道拉麵',
 '均晣溫度',
 '埔里橫濱拉麵',
 '墨洋拉麵',
 '嬡京都拉麵',
 '定置漁場三代目TheFishery',
 '宜蘭鬼匠拉麵總店',
 '屋臺拉麵',
 '希拉麵拉麵',
 '幸花拉麵雞白湯專門店',
 '彩日式拉麵專賣店',
 '御嶽海拉麵ラㄧメン',
 '惠比須麵屋',
 '招福製麵所拉麵',
 '拾貳麵舖',
 '晴拉麵',
 '暴走中村',
 '有囍yoshiramenよし拉麵',
 '木村Kimura',
 '李好拉麵大吉大利今日吃拉麵',
 '松食堂',
 '柑橘Shinn',
 '梅之香拉麵',
 '森本家拉麵',
 '極清拉麵KiyomeRamen',
 '横浜家系拉麵拉麵家',
 '次元食堂',
 '沐瀧家拉麵十二月預定',
 '泰拉風泰式拉麵',
 '涼太拉麵拉麵涼太',
 '淳鳩一夫拉麵',
 '淺草咖哩蛋包',
 '渡山樓',
 '激勵軒ChickenPowerRamen',
 '濃氣屋拉麵',
 '濟善老麵',
 '瀧澤軒食堂拉麵屋',
 '熱烈一番拉麵',
 '燈來拉麵',
 '燒燒一碗來',
 '狸匠拉麵深夜拉麵',
 '獅神拉麵',
 '琉球山日式拉麵店',
 '田中そば',
 '男子漢拉麵食堂內湖',
 '盡心亭Heartsoul',
 '目黑拉麵',
 '真劍拉麵',
 '研田拉麵',
 '竹拉麵関東清爽系豚骨',
 '籽田野菜屋',
 '織田信長',
 '美濃屋拉麵',
 '翔大拉麵店',
 '臥龍拉麵',
 '荷麵亭Hasumentei',
 '菜良

In [205]:
### DICTIONARY OF STORE NAME
dict_done = {}
for i in range(len(done)):
    dict_done[i] = done[i]
#dict_done
len(dict_done)

41

In [206]:
unique_ = []
print(len(unique))
for u in unique:
    unique_.append([u])
dict_unique = {i: unique_[j] for i,j in zip([ i for i in range(len(dict_done), len(unique)+len(dict_done))], [j for j in range(len(unique))])}
len(dict_unique)

295


295

In [207]:
dict_main_map = dict(dict_done)
dict_main_map.update(dict_unique)

In [208]:
dict_main_map

{0: ['Hirosらあ麵Kitchen', 'Hirosらあ麵Kitchen', 'Hirosらあ麵Kitchen新'],
 1: ['LeMing麵食精釀餐酒館', 'LeMing麵食精釀餐酒館'],
 2: ['Mr拉麵雲'],
 3: ['Ramen初'],
 4: ['TottoRamen鳥人拉麵', 'TottoRamen鳥人拉麵'],
 5: ['一幻拉麵臺北'],
 6: ['一本拉麵梟', '一本拉麵本店'],
 7: ['一蘭拉麵臺灣臺北本店', '一蘭拉麵臺灣臺北本店別館'],
 8: ['一風堂微風',
  '一風堂桃園',
  '一風堂微風臺',
  '一風堂中山本店',
  '一風堂新光三越店',
  '一風堂EXPRESS微',
  '一風堂EXPRESS新竹巨城店'],
 9: ['中華そば寶來軒'],
 10: ['元町家橫濱家系拉麵', '元町家橫濱家系拉麵'],
 11: ['力量拉麵', '力量拉麵仁愛'],
 12: ['叢林拉麵', '叢林拉麵'],
 13: ['太陽蕃茄拉麵',
  '太陽蕃茄拉麵',
  '太陽蕃茄拉麵左',
  '太陽蕃茄拉麵漢',
  '太陽蕃茄拉麵誠',
  '太陽蕃茄拉麵大葉',
  '太陽蕃茄拉麵站前本店',
  '太陽蕃茄拉麵EXPRESS'],
 14: ['奧特拉麵RamenUltra南', '奧特拉麵RamenUltra板'],
 15: ['富士達人日本拉麵', '富士達人日本拉麵東寧總店'],
 16: ['屯京拉麵', '屯京拉麵', '屯京拉麵忠', '屯京拉麵臺', '屯京拉麵林口', '屯京拉麵高雄夢', '屯京拉麵新光三越'],
 17: ['山嵐拉麵', '山嵐拉麵', '山嵐拉麵一', '山嵐拉麵臺灣總店'],
 18: ['山形心心', '山形心心拉麵', '山形心心拉麵嘉', '山形心心拉麵高雄', '山形心心總店臺中', '山形心心拉麵臺北站'],
 19: ['山頭火拉麵', '山頭火拉麵sogo復'],
 20: ['東京油組總本店臺北', '東京油組總本店臺北'],
 21: ['東京豚骨釜本舖'],
 22: ['樂麵屋', '樂麵屋南', '樂麵屋永'],
 23: ['橫濱家系拉麵大和家', '橫濱家系拉麵特濃屋'],
 24: ['滝禾製麵所內

In [209]:
table_1_name = []
for key, v in dict_main_map.items():
    table_1_name.append(v[0])
table_1_name = table_1_name
#table_1_name
dict_main_map

{0: ['Hirosらあ麵Kitchen', 'Hirosらあ麵Kitchen', 'Hirosらあ麵Kitchen新'],
 1: ['LeMing麵食精釀餐酒館', 'LeMing麵食精釀餐酒館'],
 2: ['Mr拉麵雲'],
 3: ['Ramen初'],
 4: ['TottoRamen鳥人拉麵', 'TottoRamen鳥人拉麵'],
 5: ['一幻拉麵臺北'],
 6: ['一本拉麵梟', '一本拉麵本店'],
 7: ['一蘭拉麵臺灣臺北本店', '一蘭拉麵臺灣臺北本店別館'],
 8: ['一風堂微風',
  '一風堂桃園',
  '一風堂微風臺',
  '一風堂中山本店',
  '一風堂新光三越店',
  '一風堂EXPRESS微',
  '一風堂EXPRESS新竹巨城店'],
 9: ['中華そば寶來軒'],
 10: ['元町家橫濱家系拉麵', '元町家橫濱家系拉麵'],
 11: ['力量拉麵', '力量拉麵仁愛'],
 12: ['叢林拉麵', '叢林拉麵'],
 13: ['太陽蕃茄拉麵',
  '太陽蕃茄拉麵',
  '太陽蕃茄拉麵左',
  '太陽蕃茄拉麵漢',
  '太陽蕃茄拉麵誠',
  '太陽蕃茄拉麵大葉',
  '太陽蕃茄拉麵站前本店',
  '太陽蕃茄拉麵EXPRESS'],
 14: ['奧特拉麵RamenUltra南', '奧特拉麵RamenUltra板'],
 15: ['富士達人日本拉麵', '富士達人日本拉麵東寧總店'],
 16: ['屯京拉麵', '屯京拉麵', '屯京拉麵忠', '屯京拉麵臺', '屯京拉麵林口', '屯京拉麵高雄夢', '屯京拉麵新光三越'],
 17: ['山嵐拉麵', '山嵐拉麵', '山嵐拉麵一', '山嵐拉麵臺灣總店'],
 18: ['山形心心', '山形心心拉麵', '山形心心拉麵嘉', '山形心心拉麵高雄', '山形心心總店臺中', '山形心心拉麵臺北站'],
 19: ['山頭火拉麵', '山頭火拉麵sogo復'],
 20: ['東京油組總本店臺北', '東京油組總本店臺北'],
 21: ['東京豚骨釜本舖'],
 22: ['樂麵屋', '樂麵屋南', '樂麵屋永'],
 23: ['橫濱家系拉麵大和家', '橫濱家系拉麵特濃屋'],
 24: ['滝禾製麵所內

In [210]:
#滝禾製麵
# '麵屋一燈Express',
# '麵屋一燈MenyaIttoTaiwan'
dict_unique

d = {'main_store': table_1_name}
df = pd.DataFrame(d)
df.index.name = 'store_id'
df.to_csv('Table model.csv')

# Table 2

In [211]:
table_2_store_id = []
for i in list(map_t2['stem_store']):
    for key, v in dict_main_map.items():
        if i in v:
            table_2_store_id.append(key)
            break
        elif (key == list(dict_main_map.keys())[-1]):
            table_2_store_id.append('9999')
            break
            

map_t2['store_id'] = table_2_store_id
map_t2 = map_t2.drop(columns=['stem_store'])

In [212]:
detail_store_id = []
table = 't2'
for i in range(len(table_2_store_id)):
    name = [table + str(i)]
    detail_store_id.append(name[0])

map_t2['detail_store_id'] = detail_store_id
detail_store_id

['t20',
 't21',
 't22',
 't23',
 't24',
 't25',
 't26',
 't27',
 't28',
 't29',
 't210',
 't211',
 't212',
 't213',
 't214',
 't215',
 't216',
 't217',
 't218',
 't219',
 't220',
 't221',
 't222',
 't223',
 't224',
 't225',
 't226',
 't227',
 't228',
 't229',
 't230',
 't231',
 't232',
 't233',
 't234',
 't235',
 't236',
 't237',
 't238',
 't239',
 't240',
 't241',
 't242',
 't243',
 't244',
 't245',
 't246',
 't247',
 't248',
 't249',
 't250',
 't251',
 't252',
 't253',
 't254',
 't255',
 't256',
 't257',
 't258',
 't259',
 't260',
 't261',
 't262',
 't263',
 't264',
 't265',
 't266',
 't267',
 't268',
 't269',
 't270',
 't271',
 't272',
 't273',
 't274',
 't275',
 't276',
 't277',
 't278',
 't279',
 't280',
 't281',
 't282',
 't283',
 't284',
 't285',
 't286',
 't287',
 't288',
 't289',
 't290',
 't291',
 't292',
 't293',
 't294',
 't295',
 't296',
 't297',
 't298',
 't299',
 't2100',
 't2101',
 't2102',
 't2103',
 't2104',
 't2105',
 't2106',
 't2107',
 't2108',
 't2109',
 't2110',


In [214]:
map_t2 = map_t2[['detail_store_id','store_id','store', 'address', 'description', 'open_time', 'latitude', 'longtitute',
       'map_review', 'region', 'soup', 'transport']]
map_t2.to_csv('Store.csv', index=False)

In [ ]:
len(list(map_t2['store_id']))

In [ ]:
for i in range(len(map_t2['store_id'])):
    print('t2' + str(i))

In [ ]:
for i in range(len(table_2_store_id)):
    name = ['t2' + str(i)]
    detail_store_id.append(name[0])

In [ ]:
detail_store_id

In [ ]:
for i in range(len(table_2_store_id)):
    name = 't2' + str(i)
    detail_store_id.append([name][0])

In [ ]:
table_2_store_id